In [11]:
# 1. 문제 정의
# 평가: R^2
# target: Price
# 최종 파일: result.csv(컬럼 1개 pred, 1 확률값)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("laptop_train.csv")
test = pd.read_csv("laptop_test.csv")

# 3. 탐색적 데이터 분석(EDA)
print("===== 데이터 크기 =====")
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)

print("\n===== 데이터 정보(자료형) =====")
print(train.info())
      
print("\n===== train 결측치 수 =====")
print(train.isnull().sum())

print("\n===== test 결측치 수 =====")
print(test.isnull().sum())

print("\n===== 카테고리 비교 =====")
cols = train.select_dtypes(include='object').columns
for col in cols:
    set_train = set(train[col])
    set_test = set(test[col])
    same = (set_train == set_test)
    if same:
        print(col, "\t카테코리 동일함")
    else:
        print(col, "\t카테고리 동일하지 않음")

print("\n===== target 기술 통계 =====")
print(train['Price'].describe())

===== 데이터 크기 =====
Train Shape: (91, 10)
Test Shape: (39, 9)

===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Brand               91 non-null     object 
 1   Model               82 non-null     object 
 2   Series              55 non-null     object 
 3   Processor           86 non-null     object 
 4   Processor_Gen       86 non-null     object 
 5   RAM                 85 non-null     float64
 6   Hard_Disk_Capacity  85 non-null     object 
 7   OS                  85 non-null     object 
 8   Rating              91 non-null     float64
 9   Price               91 non-null     int64  
dtypes: float64(2), int64(1), object(7)
memory usage: 7.2+ KB
None

===== train 결측치 수 =====
Brand                  0
Model                  9
Series                36
Processor              5
Processor_Gen          5
RAM   

In [13]:
# 4. 데이터 전처리
target = train.pop('Price')

# 결측치 처리(범주형)
c_cols = ['Model', 'Series', 'Processor', 'Processor_Gen', 'Hard_Disk_Capacity', 'OS']
train[c_cols] = train[c_cols].fillna("X")
test[c_cols] = test[c_cols].fillna("X")

# 결측치 처리(연속형)
n_cols = ['RAM']
train[n_cols] = train[n_cols].fillna(-1)
test[n_cols] = test[n_cols].fillna(-1)

# 원-핫 인코딩
combined = pd.concat([train, test])
combined_dummies = pd.get_dummies(combined)
n_train = len(train)
train = combined_dummies[:n_train]
test = combined_dummies[n_train:]

# 5. 검증 데이터 나누기
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

from sklearn.metrics import r2_score
result = r2_score(y_val, pred)
print('\n r2:', result)

# 7. 예측 및 결과 파일 생성
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv", index=False)

print("\n===== 제출 파일(샘플 5개) =====")
print(pd.read_csv("result.csv").head())


 r2: 0.7496764602229047

===== 제출 파일(샘플 5개) =====
       pred
0  40979.15
1  39366.98
2  60343.56
3  39960.00
4  44331.60


In [15]:
# 성능 개선

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("laptop_train.csv")
test = pd.read_csv("laptop_test.csv")

# 4. 데이터 전처리
target = train.pop('Price')

# 결측치 삭제
train = train.drop('Series', axis=1)
test = test.drop('Series', axis=1)
train = train.drop('Model', axis=1)
test = test.drop('Model', axis=1)

# 결측치 처리(범주형)
c_cols = ['Processor', 'Processor_Gen', 'Hard_Disk_Capacity', 'OS']
train[c_cols] = train[c_cols].fillna("X")
test[c_cols] = test[c_cols].fillna("X")

# 결측치 처리(연속형)
n_cols = ['RAM']
train[n_cols] = train[n_cols].fillna(-1)
test[n_cols] = test[n_cols].fillna(-1)

# 원-핫 인코딩
combined = pd.concat([train, test])
combined_dummies = pd.get_dummies(combined)
n_train = len(train)
train = combined_dummies[:n_train]
test = combined_dummies[n_train:]

# 5. 검증 데이터 나누기
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

from sklearn.metrics import r2_score
result = r2_score(y_val, pred)
print('\n r2:', result)

# 7. 예측 및 결과 파일 생성
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv", index=False)


 r2: 0.8042392429064131
